In [1]:
import sys
sys.setrecursionlimit(6000)
import pickle
from sklearn.metrics import accuracy_score, precision_score, classification_report, confusion_matrix
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from nltk.corpus import stopwords
import nltk
import re
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np
import pandas as pd
from IPython.display import display
import joblib

In [2]:
class TextSelector(BaseEstimator, TransformerMixin):
    def __init__(self, field):
        self.field = field
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.field]
class NumberSelector(BaseEstimator, TransformerMixin):
    def __init__(self, field):
        self.field = field
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[[self.field]]


In [3]:
def Tokenizer(str_input):
    words = re.sub(r"[^A-Za-z0-9\-]", " ", str_input).lower().split()
    #porter_stemmer=nltk.PorterStemmer()
    #words = [porter_stemmer.stem(word) for word in words]
    word_lemma= nltk.WordNetLemmatizer()
    words=[word_lemma.lemmatize(word) for word in words]
    return words

In [ ]:
"""
stop_words = set(stopwords.words('english')) 
classifier = Pipeline([
    ('features', FeatureUnion([
        ('text', Pipeline([
            ('colext', TextSelector('Text')),
            ('tfidf', TfidfVectorizer(tokenizer=Tokenizer, stop_words=stop_words,
                     min_df=.0025, max_df=0.25, ngram_range=(1,3))),
            ('svd', TruncatedSVD(algorithm='randomized', n_components=300)), #for XGB
        ])),
        ('words', Pipeline([
            ('wordext', NumberSelector('TotalWords')),
            ('wscaler', StandardScaler()),
        ])),
    ])),
    ('clf', XGBClassifier(max_depth=4, n_estimators=500,nthread =-1, learning_rate=0.1, scale_pos_weight=0.1)),
#    ('clf', RandomForestClassifier()),
    ])
"""

In [ ]:
"""
classifier.fit(X_train, y_train)
preds = classifier.predict(X_test)
"""

In [ ]:
"""
Accuracy: 0.9323415426928366
Precision: 0.9902676399026764
              precision    recall  f1-score   support

           0       0.93      1.00      0.96     36080
           1       0.99      0.31      0.47      3915

    accuracy                           0.93     39995
   macro avg       0.96      0.66      0.72     39995
weighted avg       0.94      0.93      0.92     39995

[[36068    12]
 [ 2694  1221]]
"""

In [4]:
mymodel=open('model.pkl','rb')
pipe=joblib.load(mymodel)

In [5]:
def Filter(Text, Totalwords):
    data = [[Text, Totalwords]]
    df1 = pd.DataFrame(data,columns = ['Text', 'TotalWords'])
    predict = pipe.predict_proba(df1)[:,1] >= 0.03
    predict
    return predict
  

In [6]:
from tkinter import *
from tkinter import ttk

In [8]:
def GUI_function(self):
    user_input = comment.get() 
    predict= Filter(user_input,50)
    
    if predict == 0:
        result.delete(0, "end")
        result.insert(0,"Text is fine")
    else:
        result.delete(0, "end")
        result.insert(0,"Comment can't be posted due to inappropriate language")

def clear_screen():
    result.delete(0, "end")
    result.insert(0," ")
    comment.delete(0, "end")
    comment.insert(0," ")
    
def quitApplication():
    main.destroy()     
    

main = Tk()
main.title('Online Abusive Comment Detector')

frame = Frame(main)
frame.grid()

label1 = Label(frame, text="Post a Comment")
label1.grid(row=0, column=20, padx=4)
comment = Entry(frame, width=60)
comment.grid(row=0, column=21, padx=4)
comment.focus()

button_check = Button(frame,text='post')
button_check.bind("<Button>",GUI_function)
button_check.grid(row=2, column=44)

button_clear = Button(frame, text='clear screen',command=clear_screen)
button_clear.grid(row=2, column=10)

button_quit = Button(frame, text='Quit',command=quitApplication)
button_quit.grid(row=3, column=10)


label2 = Label(frame, text="Result")
label2.grid(row=4, column=20, padx=4)
result = Entry(frame,width=60)
result.grid(row=4, column=21, padx=4)

 
main.mainloop()